In [1]:
import re
from copy import deepcopy

In [6]:
class State:
    def __init__(self, blueprint, time):
        self.ore_robots = 1
        self.clay_robots = 0
        self.obsidian_robots = 0
        self.geode_robots = 0
        self.ore = 0
        self.clay = 0
        self.obsidian = 0
        self.geode = 0
        self.time = time
        self.blueprint = blueprint
        self.max_ore = max(m.get('ore', 0) for m in self.blueprint.values())
        self.max_clay = max(m.get('clay', 0) for m in self.blueprint.values())
        self.max_obsidian = max(m.get('obsidian', 0) for m in self.blueprint.values())

    def collect_ore(self):
        self.ore += self.ore_robots

    def collect_clay(self):
        self.clay += self.clay_robots

    def collect_obsidian(self):
        self.obsidian += self.obsidian_robots

    def collect_geode(self):
        self.geode += self.geode_robots

    def build_ore_robot(self):
        self.ore -= self.blueprint['ore_robot']['ore']
        self.ore_robots += 1

    def build_clay_robot(self):
        self.ore -= self.blueprint['clay_robot']['ore']
        self.clay_robots += 1

    def build_obsidian_robot(self):
        self.ore -= self.blueprint['obsidian_robot']['ore']
        self.clay -= self.blueprint['obsidian_robot']['clay']
        self.obsidian_robots += 1
    
    def build_geode_robot(self):
        self.ore -= self.blueprint['geode_robot']['ore']
        self.obsidian -= self.blueprint['geode_robot']['obsidian']
        self.geode_robots += 1

    def can_build_ore_robot(self):
        return self.ore >= self.blueprint['ore_robot']['ore']

    def can_build_clay_robot(self):
        return self.ore >= self.blueprint['clay_robot']['ore']

    def can_build_obsidian_robot(self):
        return self.ore >= self.blueprint['obsidian_robot']['ore'] and self.clay >= self.blueprint['obsidian_robot']['clay']

    def can_build_geode_robot(self):
        return self.ore >= self.blueprint['geode_robot']['ore'] and self.obsidian >= self.blueprint['geode_robot']['obsidian']

    def collect(self):
        self.collect_ore()
        self.collect_clay()
        self.collect_obsidian()
        self.collect_geode()
        self.time -= 1

    def get_state(self):
        return (self.ore, self.clay, self.obsidian, self.geode, self.ore_robots, 
                self.clay_robots, self.obsidian_robots, self.geode_robots)

    def __repr__(self):
        return (f'Ore: {self.ore}, Clay: {self.clay}, Obsidian: {self.obsidian}, Geode: {self.geode}\n'
        + f'Ore robots: {self.ore_robots}, Clay robots: {self.clay_robots}, Obsidian robots: {self.obsidian_robots}, Geode robots: {self.geode_robots}\n'
        + f'Time: {self.time}')


def read_input(filename):
    expr = r'Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian.'
    blueprints = {}
    with open(filename, 'r') as f:
        for line in f.readlines():
            m = re.search(expr, line.strip())
            blueprints[int(m[1])] = {
                'ore_robot': {'ore': int(m[2])},
                'clay_robot': {'ore': int(m[3])},
                'obsidian_robot': {'ore': int(m[4]), 'clay': int(m[5])},
                'geode_robot': {'ore': int(m[6]), 'obsidian': int(m[7])}
            }
    return blueprints

def get_next_states(state):
    states = []
    new_state = deepcopy(state)
    new_state.collect()
    states.append(new_state)
    if state.can_build_ore_robot():
        new_state = deepcopy(state)
        new_state.collect()
        new_state.build_ore_robot()
        states.append(new_state)
    if state.can_build_clay_robot():
        new_state = deepcopy(state)
        new_state.collect()
        new_state.build_clay_robot()
        states.append(new_state)
    if state.can_build_obsidian_robot():
        new_state = deepcopy(state)
        new_state.collect()
        new_state.build_obsidian_robot()
        states.append(new_state)
    if state.can_build_geode_robot():
        new_state = deepcopy(state)
        new_state.collect()
        new_state.build_geode_robot()
        states.append(new_state)
    return states

def keep_state(prev, curr, max_geodes):
    if curr.ore_robots > curr.max_ore and curr.clay_robots > curr.max_clay and curr.obsidian_robots > curr.max_obsidian:
        return False
    if curr.ore > curr.max_ore * curr.time or curr.clay > curr.max_clay * curr.time or curr.obsidian > curr.max_obsidian * curr.time:
        return False
    if curr.time * curr.geode_robots < max_geodes:
        return False
    return True

def dfs(start_state):
    s = [start_state]
    explored = set()
    max_geodes = 0
    while s:
        v = s.pop()
        if v.time == 0:
            if v.geode > max_geodes:
                max_geodes = v.geode
            continue
        for state in get_next_states(v):
            if state.get_state() not in explored and keep_state(v, state, max_geodes):
                s.append(state)
                explored.add(state.get_state())
    return max_geodes


In [7]:
blueprints = read_input('19_input.txt')

In [8]:
blueprint_geodes = []
for (i, blueprint) in blueprints.items():
    start_state = State(blueprint, 24)
    max_geodes = dfs(start_state)
    blueprint_geodes.append(max_geodes)
    print(f'Blueprint {i}: {max_geodes}')

Blueprint 1: 0
Blueprint 2: 0


In [44]:
sum(i * j for (i, j) in enumerate(blueprint_geodes, 1))

2144